In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

: 

In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
df.info()

In [ ]:
#ID 1차 전처리 = cabin, last_name, homeplant, destination
df['last_name'] = df['Name'].str.split().str[-1]
df['id'] = df.PassengerId.str.split('_').str[0]
df['HomePlanet'] = df.groupby('id')['HomePlanet'].fillna(method='ffill')
df['last_name'] = df.groupby('id')['last_name'].fillna(method='ffill')
df['Cabin'] = df.groupby('id')['Cabin'].fillna(method='ffill')

#cryosleep일때 int데이터 0
#돈을 사용했다 = 동면이 아님 / 돈을 안썼다 = 동면이다
df['price'] = df['RoomService'] + df['FoodCourt'] + df['ShoppingMall'] + df['Spa'] + df['VRDeck']
df['price'].fillna(0, inplace=True)
df['CryoSleep'] = np.where(df['price'] > 0, False, df['CryoSleep'])
df['CryoSleep'].fillna(True, inplace=True)
df.loc[df['CryoSleep'] == True, ['RoomService','FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = 0

In [ ]:
# last_name을 통한 homePlanet
# Cabin이 null이 아닌 행에서만 last_name을 ffill.bfill로 채움
mask = df['Cabin'].notna()
df.loc[mask, 'last_name'] = df[mask].groupby('Cabin')['last_name'].transform(lambda x: x.ffill().bfill())
df['HomePlanet'] = df.groupby('last_name')['HomePlanet'].transform(lambda x: x.ffill().bfill())

In [ ]:
df.groupby(['last_name','HomePlanet']).count()

In [ ]:
import seaborn as sns
sns.pairplot(df, hue = 'Transported')

In [ ]:
#EDA를 이용한 수치형 데이터 분석
#일원산분석 // transported에 따른 int값 분석
import scipy.stats as stats
# 각 수치형 변수에 대해 ANOVA 수행
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_columns:
    if col != 'Transported':
        groups = [df[df['Transported']==i][col].dropna() for i in [0,1]]
        f_stat, p_val = stats.f_oneway(*groups)
        print(f"{col}: F-statistic={f_stat}, p-value={p_val}")
#shoppingmall을 제외하곤 분산 유사성 없음

In [ ]:
#cabin에 따른 homeplanet 결측치 채우기 // DEF는 불명
df['Cabin_deck'] = df['Cabin'].str[0]
df.loc[(df['HomePlanet'].isnull()) & (df['Cabin_deck'].isin(['A','B','C','T'])), 'HomePlanet'] = 'Europa'
df.loc[(df['HomePlanet'].isnull()) & (df['Cabin_deck'] == 'G'), 'HomePlanet'] = 'Earth'

In [ ]:
# cabin def 분류 // 현재랑 비슷하게
print(df.groupby(['Cabin_deck','HomePlanet'])['HomePlanet'].count())
df[df['HomePlanet'].isnull() & df['Cabin_deck'].isin(['D','E','F'])] .groupby('Cabin_deck').size()


In [ ]:
# 비율대로 채우기 
ratio = df.dropna(subset=['HomePlanet']).groupby('Cabin_deck')['HomePlanet'].value_counts(normalize=True)
def fill_homeplanet(row):
    if pd.isnull(row['HomePlanet']) and row['Cabin_deck'] in ['D','E','F']:
        probs = ratio[row['Cabin_deck']]
        return np.random.choice(probs.index, p=probs.values)
    else:
        return row['HomePlanet']
df['HomePlanet'] = df.apply(fill_homeplanet, axis=1)

In [ ]:
sns.countplot(data=df, x='Destination')

In [ ]:
print('test')